## Argomenti affrontati

-   Svolgimento di un tema d'esame di Statistica I
-   Modello di regressione lineare semplice

::: callout-note
#### Nota

Gli esercizi **R** associati sono disponibili a [questo link](https://tommasorigon.github.io/introR/exe/es_2.html)
:::

## Descrizione del problema

Per $n = 17$ luoghi nelle Alpi viene misurata la **pressione atmosferica** (inHg, ovvero "*inches of mercury*") e la **temperatura di ebollizione** dell'acqua (in gradi Fahrenheit).

I dati provengono da un esperimento condotto dal **fisico scozzese Forbes** nel 1857.

Forbes era interessato a stimare l'altitudine tramite la pressione. Tuttavia, il barometro all'epoca era uno strumento pesante e costoso.

In montagna infatti l'acqua bolle ad una temperatura diversa, per cui è possibile cercare di stimare la pressione a partire dalla temperatura di ebollizione.

::: callout-note
#### Nota

I dati seguenti sono gli stessi dell'esame di Statistica I del 11 Novembre 2020, che trovate sul sito web.
:::


## Importazione dei dati `forbes`

Come fatto in precedenza, anzitutto è necessario scaricare il file `forbes.csv` e salvarlo nel proprio computer. [Link al file](../dataset/forbes.csv)

In [ ]:
forbes <- read.table("../dataset/forbes.csv", header = TRUE, sep = ",")

In alternativa, possiamo semplice ottenerli usando il link:

In [ ]:
#| eval: false
path <- "https://tommasorigon.github.io/introR/data/forbes.csv"
forbes <- read.table(path, header = TRUE, sep = ",")

In [ ]:
str(forbes)

## Operazioni preliminari

Per motivi **interpretativi**, convertiamo la temperatura da gradi Farenheit a gradi Celsius, ricordando che

$$
(\text{``Fahrenheit''}) = 32 + \frac{9}{5}(\text{``Celsius''}).
$$

In [ ]:
colnames(forbes) <- c("TempF", "Pressione") # Cambio i nomi alle variabili

forbes$TempC <- round((forbes$TempF - 32) * 5 / 9, 2) # Da Fahrenheit a Celsius
summary(forbes)

Come mai approssimiamo i valori utilizzando `round`? Per motivi **estetici**: in questo modo si ottengono risultati identici alla prova d'esame di Statistica I.

## Istogramma

::: callout-warning
#### Domanda 1

Si disegni un istogramma della variabile temperatura, scegliendo un numero appropriato di classi equispaziate e giustificandone la scelta.

Si aggiungano a questi grafici gli "abbellimenti" grafici ritenuti necessari (nomi delle variabili, titolo, etc).
:::

#### Soluzione

Possiamo decidere di specificare in autonomia gli intervalli delle classi oppure di lasciare ad **R** questa scelta.

In [ ]:
par(mfrow = c(1, 2)) # Divido la finestra grafica in 2 parti

# Opzione 1, per un totale di 6 classi equispaziate
hist(forbes$TempC) # Equivalente a: hist(forbes$TempC, breaks = "sturges") 

# Opzione 2, definisco manualmente 5 classi equispaziate
breaks <- c(90, 92.5, 95, 97.5, 100, 102.5)
hist(forbes$TempC, breaks = breaks)

La soluzione di sinistra fa uso di $6$ classi. Viceversa, quella di sinistra fa uso di $5$ classi, come nella **soluzione** dell'esame.

## Indici di posizione

::: callout-warning
#### Domanda 2

Si ottengano la media aritmetica di entrambe le variabili `tempC` e `pressione`. Quanto vale la temperatura di ebollizione media espressa in gradi Fahrenheit? Si risponda senza calcolare tutti i valori della variabile `tempF`.
:::

### Soluzione

Abbiamo già calcolato la medie tramite il comando `summary`, per completezza:

In [ ]:
# Prima parte della domanda
mean(forbes$TempC)
mean(forbes$Pressione)

In [ ]:
# Seconda parte della domanda 
32 + 9 / 5 * mean(forbes$TempC) # Utilizzo proprietà della media
mean(forbes$TempF) # Non richiesto, calcola la media a partire dai dai dati trasformati

::: callout-note
#### Esercizio

Come mai le medie calcolate nella seconda parte differiscono leggermente? A cosa può essere dovuto?
:::

## Indici di variabilità

::: callout-warning
#### Domanda 3

Si ottenga la varianza delle variabili `tempC` e `pressione`.
:::

#### Soluzione

Dato che tornerà utile in seguito, definiamo la funzione `my_var` che calcola la **varianza**.

In [ ]:
# Si, la funzione è definita in un'unica riga e non c'è nulla di male in questo
my_var <- function(x) mean(x^2) - mean(x)^2 

# Calcolo delle due varianze
my_var(forbes$TempC)
my_var(forbes$Pressione)

::: callout-note
#### Esercizio

Si ottengano i momenti secondi delle variabili `tempC` e `pressione`.
:::

## Covarianza e correlazione

::: callout-warning
## Domanda 4

Si disegni un opportuno grafico che aiuti a comprendere la relazione tra le due variabili. Si calcoli quindi la correlazione.
:::

#### Soluzione

Per la prima parte della domanda, abbiamo bisogno del nuovo comando `plot`, che può essere usato (tra le altre cose!) per costruire un **diagramma a dispersione**.

Forniamo due versioni dello stesso grafico; la seconda contiene dei miglioramenti estetici.

In [ ]:
par(mfrow = c(1, 1)) # Vogliamo mostrare un grafico alla volta
plot(forbes$TempC, forbes$Pressione)
plot(forbes$TempC, forbes$Pressione, pch = 16, xlab = "Temperatura", ylab = "Pressione")

È quindi evidente che i dati siano circa (anche se non perfettamente) allineati

Per la seconda parte di domanda (correlazione), dobbiamo anzitutto ottenere la **covarianza** tra due variabili.

La **covarianza** tra due insiemi di dati $x_1,\dots,x_n$ e $y_1,\dots,y_n$ è definita come

$$
\text{cov}(x,y) = \frac{1}{n}\sum_{i=1}^n(x_i - \bar{x})(y_i - \bar{y}) = \frac{1}{n}\sum_{i=1}^nx_i y_i - \bar{x}\bar{y}.
$$

Definiamo quindi la funzione `my_cov`, che calcola appunto la covarianza:

In [ ]:
my_cov <- function(x, y) mean(x * y) - mean(x) * mean(y)
my_cov(forbes$TempC, forbes$Pressione) # = my_cov(forbes$Pressione, forbes$TempC)

In **R** esiste anche il comando `cov` che, come nel caso della varianza, divide la sommatoria per $(n - 1)$ e non $n$ per motivi legati all'**inferenza statistica**:

In [ ]:
cov(forbes$Pressione, forbes$TempC) # = 17 / 16 * my_cov(forbes$TempC, forbes$Pressione)

L'indice di \empb{correlazione} è definito come: $$
\rho = \frac{\text{cov}(x,y)}{\sqrt{\text{var}(x) \text{var}(y)}}.
$$

Pertanto, possiamo calcolare la correlazione nei modo seguente:

In [ ]:
my_cov(forbes$TempC, forbes$Pressione) / sqrt(my_var(forbes$TempC) * my_var(forbes$Pressione))

cov(forbes$TempC, forbes$Pressione) / sqrt(var(forbes$TempC) * var(forbes$Pressione))

::: callout-note
#### Esercizio

Come mai i risultati dei due comandi coincidono? Si verifichi questo fatto svolgendo **analiticamente** (carta e penna) i conti.
:::

In **R** esiste anche il comando `cor`, che permette di ottenere la correlazione

In [ ]:
correlation <- cor(forbes$TempC, forbes$Pressione)
correlation

## Modello di regressione lineare

::: callout-warning
## Domanda 5

Si ottenga la retta ai minimi quadrati per la relazione tra `tempC` e `pressione` e la si disegni nel grafico ottenuto in precedenza.
:::

#### Soluzione

Anzitutto ricordiamo che in un **modello lineare** del tipo $y_i = \alpha + \beta x_i + \epsilon_i$, le stime ai **minimi quadrati** sono pari a $$
\hat{\alpha} = \bar{y} - \hat{\beta}\:\bar{x}, \qquad \hat{\beta}  = \frac{\text{cov}(x,y)}{\text{var}(x)}.
$$

Pertanto, possiamo calcolare la correlazione nei modo seguente:

In [ ]:
# Coefficiente angolare
beta_hat <- my_cov(forbes$TempC, forbes$Pressione) / my_var(forbes$TempC)
# Intercetta
alpha_hat <- mean(forbes$Pressione) - mean(forbes$TempC) * beta_hat

c(alpha_hat, beta_hat)

In [ ]:
plot(forbes$TempC, forbes$Pressione, pch = 16, xlab = "Temperatura", ylab = "Pressione")
abline(a = alpha_hat, b = beta_hat)

::: callout-warning
## Domanda 6

In base al modello stimato, se la temperatura di ebollizione dell'acqua è pari 97 gradi Celsius, a quanto è pari la pressione?
:::

#### Soluzione

Utilizzando le stime ottenute, possiamo calcolare rapidamente i valori previsti:

In [ ]:
x <- seq(from = 90, to = 100, length = 20)
alpha_hat + beta_hat * x

In particolare, quando `tempC = 97` si ha che:

In [ ]:
alpha_hat + beta_hat * 97

::: callout-warning
## Domanda 7

Si ottenga un indice di bontà di adattamento ai dati della curva ottenuta.
:::

#### Soluzione

Il coefficiente $R^2$ per un modello di regressione lineare semplice è definito come: $$
R^2 = 1 - \frac{\text{var}(r)}{\text{var}(y)} = \rho^2,
$$ dove $r_1,\dots,r_n$ sono i **residui**.

Anzitutto quindi calcoliamo i residui:

In [ ]:
residuals <- forbes$Pressione - (alpha_hat + beta_hat * forbes$TempC)

Il coefficiente $R^2$ può quindi essere ottenuto in due modi diversi:

In [ ]:
correlation^2
1 - my_var(residuals) / my_var(forbes$Pressione)

## Esercizio riassuntivo

Si consideri l'esame di Statistica I del 28 Gennaio 2021, disponibile a [questo link](https://tommasorigon.github.io/StatI/esami/28_01_2021.html)

Si risolva l'esercizio 2 dell'esame usando il software **R**.

::: callout-tip
#### Suggerimento

Per poter importare piccole quantità di dati in **R**, è possibile usare il comando `scan`.
:::